In [1]:
import os

from tqdm.notebook import trange, tqdm
from p_tqdm import p_imap

from utils.pdf_processing import get_all_data
import psycopg2
from psycopg2.extensions import register_adapter
from psycopg2.extras import Json

In [2]:
register_adapter(dict, Json)

conn = psycopg2.connect("dbname=collegepdf user=postgres")

cur = conn.cursor()

### Delete records in DB

In [3]:
sql_1 = "DROP TABLE IF EXISTS textract;"
sql_2 = r"""
CREATE TABLE textract (
  id SERIAL PRIMARY KEY,
  filepath VARCHAR(255) NOT NULL,
  filename VARCHAR(255) NOT NULL,
  college VARCHAR (255) NOT NULL,
  pdfminer TEXT,
  pdfminer_detailed JSON[],
  fonts JSON,
  pypdf2 TEXT,
  pypdf2_detailed JSON[]
);
""".replace('\n', '')

for sql in [sql_1, sql_2]:
    cur.execute(sql)

sql_3 = """
INSERT INTO textract 
  (filepath, filename, college, pdfminer, pdfminer_detailed, pypdf2) 
  VALUES (%(filepath)s, %(filename)s, %(college)s, %(pdfminer)s, %(pdfminer_detailed)s::json[], %(pypdf2)s)
""".replace('\n', '')

In [4]:
conn.commit()

### Append records to DB

In [5]:
batch_1 = ['Arkansas', 'Bowdoin', 'Ohio State']
batch_2 = ['Colorado - Boulder', 'Minnesota Twin Cities', 'UCLA', 'U Conn']
limit_folders = batch_1 + batch_2
# limit_folders = []

files = [directory + '/' + file
         for directory, _, files in list(os.walk('data'))
         for file in files
         if '.pdf' in file
         and (directory.split('/')[1] in limit_folders
              if len(limit_folders) != 0
              else True)]

fn = lambda path: get_all_data(path)

# # multiprocessing + tqdm
iterator = p_imap(fn, files)

for result in iterator:
    try:
        cur.execute(sql_3, result)
        print(result['filename'])
    except Exception as e:
        print(f, e)

conn.commit()



/home/marcin/.local/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='data/UCLA/UCLA_2001–03.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)
/home/marcin/.local/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='data/UCLA/UCLA_2007–08.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)
/home/marcin/.local/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='data/UCLA/UCLA_2009–10.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warning

UCLA_2017–18.pdf
UCLA_2009–10.pdf
UCLA_1997–99.pdf
UCLA_2007–08.pdf
UCLA_1995–97.pdf


UCLA_2013–14.pdf
UCLA_2001–03.pdf
UCLA_2008–09.pdf
UCLA_2010–11.pdf
UCLA_2016–17.pdf
UCLA_2011–12.pdf


UCLA_2019–20.pdf


UCLA_2018–19.pdf
UCLA_1992–93.pdf
UCLA_2003–05.pdf
UCLA_2012–13.pdf


UCLA_1994–95.pdf


UCLA_2005–07.pdf
UCLA_2014–15.pdf
UCLA_1991–92.pdf
UCLA_2015–16.pdf
UCLA_1993–94.pdf
UCLA_1999–2001.pdf
U Conn_2008-2009.pdf
U Conn_2010-2011.pdf
U Conn_2007-2008.pdf
U Conn_2015-2016.pdf
U Conn_2016-2017.pdf
U Conn_2014-2015.pdf
U Conn_2006-2007.pdf
U Conn_2017-2018.pdf
U Conn_1998-1999.pdf
U Conn_2005-2006.pdf
U Conn_2000-2001.pdf
U Conn_2018-2019.pdf
U Conn_2012-2013.pdf
U Conn_2002-2003.pdf
U Conn_2003-2004.pdf
U Conn_1999-2000.pdf


U Conn_2011-2012.pdf
U Conn_2009-2010.pdf
U Conn_2001-2002.pdf
U Conn_2004-2005.pdf


U Conn_2019-2020.pdf
U Conn_2013-2014.pdf
Bowdoin_2014-2015.pdf
Bowdoin_2016-2017.pdf
Bowdoin_2015-2016.pdf
Bowdoin_2019-2020.pdf
Bowdoin_2012-2013.pdf
Bowdoin_2018-2019.pdf
Bowdoin_2013-2014.pdf
Bowdoin_2010-2011.pdf
Bowdoin_2011-2012.pdf
Bowdoin_2009-2010.pdf
Bowdoin_2017-2018.pdf
Bowdoin_2019-2020.pdf
Bowdoin_2016-2017.pdf
Bowdoin_2010-2011.pdf
Bowdoin_2013-2014.pdf
Bowdoin_2009-2010.pdf
Bowdoin_2015-2016.pdf
Bowdoin_2017-2018.pdf
Bowdoin_2018-2019.pdf
Bowdoin_2012-2013.pdf
Bowdoin_2011-2012.pdf
Bowdoin_2014-2015.pdf
Arkansas_2017-2018.pdf
Arkansas_2002-2003.pdf
Arkansas_2012-2013.pdf
Arkansas_2010-2011.pdf
Arkansas_2011-2012.pdf
Arkansas_2015-2016.pdf
Arkansas_2007-2008.pdf
Arkansas_2016-2017.pdf
Arkansas_2013-2014.pdf
Arkansas_2003-2004.pdf
Arkansas_2018-2019.pdf
Arkansas_2014-2015.pdf
Arkansas_2009-2010.pdf
Arkansas_2006-2007.pdf
Arkansas_2019-2020.pdf
Arkansas_2008-2009.pdf
Arkansas_2001-2002.pdf
Arkansas_2004-2005.pdf
Arkansas_2014-2015.pdf
Arkansas_2016-2017.pdf
Arkansas_2018-

Arkansas_2019-2020.pdf
Arkansas_2015-2016.pdf
Arkansas_2020-2021.pdf
Arkansas_2017-2018.pdf
Arkansas_2020-2021.pdf
Ohio State_2016-2017.pdf
Ohio State_2014-2015.pdf
Ohio State_2009-2010.pdf
Ohio State_2020-2021.pdf
Ohio State_2006-2007.pdf
Ohio State_2019-2020.pdf
Ohio State_2017-2018.pdf
Ohio State_2013-2014.pdf
Ohio State_2008-2009.pdf
Ohio State_2018-2019.pdf
Ohio State_2005-2006.pdf
Ohio State_2012-2013.pdf
Ohio State_2010-2011.pdf
Ohio State_2015-2016.pdf
Ohio State_2007-2008.pdf
Colorado - Boulder_1995-1996.pdf
Colorado - Boulder_2009-2010.pdf
Colorado - Boulder_2002-2003.pdf
Colorado - Boulder_1989-1990.pdf
Colorado - Boulder_2000-2001.pdf
Colorado - Boulder_2004-2005.pdf
Colorado - Boulder_2003-2004.pdf
Colorado - Boulder_1994-1995.pdf
Colorado - Boulder_2005-2006.pdf
Colorado - Boulder_2017-2018.pdf
Colorado - Boulder_1993-1994.pdf
Colorado - Boulder_2016-2017.pdf
Colorado - Boulder_1991-1992.pdf
Colorado - Boulder_2001-2002.pdf
Colorado - Boulder_2007-2008.pdf
Colorado - Boul

In [6]:
files = [directory + '/' + file
         for directory, _, files in list(os.walk('data'))
         for file in files
         if '.pdf' in file
         and (directory.split('/')[1] not in limit_folders
              if len(limit_folders) != 0
              else True)]

fn = lambda path: get_all_data(path)

# # multiprocessing + tqdm
iterator = p_imap(fn, files)

for result in iterator:
    try:
        cur.execute(sql_3, result)
        print(result['filename'])
    except Exception as e:
        print(f, e)
conn.commit()


Iowa_2006-20.pdf


Iowa_2018-19.pdf
Iowa_1978-80.pdf
Iowa_2007-08.pdf
Iowa_1998-20.pdf
Iowa_1986-88.pdf
Iowa_1996-98.pdf
Iowa_1990-92.pdf
Iowa_1980-82.pdf
Iowa_,1976-7.pdf
Iowa_2002-20.pdf
Iowa_2000-20.pdf
Iowa_2019-20.pdf
Iowa_1984-86.pdf
Iowa_2016-17.pdf
Iowa_2015-16.pdf
Iowa_1994-96.pdf
Iowa_2014-15.pdf
Iowa_2010-11.pdf
Iowa_2011-12.pdf
Iowa_2013-14.pdf
Iowa_2009-10.pdf
Iowa_2005-20.pdf
Iowa_1982-84.pdf
Iowa_2012-13.pdf
Iowa_,1972-7.pdf
Iowa_1992-94.pdf
Iowa_2004-20.pdf
Iowa_2008-09.pdf
Iowa_1970-19.pdf
Iowa_1988-90.pdf
Iowa_,1974-7.pdf
Iowa_2017-18.pdf
Duke_2001.pdf
Duke_2010.pdf
Duke_2007.pdf
Duke_2015.pdf
Duke_2011.pdf
Duke_2002.pdf
Duke_2014.pdf
Duke_2020.pdf
Duke_2013.pdf
Duke_2000.pdf
Duke_2012.pdf
Duke_2008.pdf
Duke_1999.pdf
Duke_2018.pdf
Duke_2003.pdf
Duke_2009.pdf
Duke_2006.pdf
Duke_2004.pdf
Duke_2019.pdf
Duke_2016.pdf
Duke_2005.pdf
Duke_2017.pdf
Colby_2016-2017.pdf
Colby_2017-2018.pdf
Colby_2012-2013.pdf
Colby_2020-2021.pdf
Colby_2018-2019.pdf
Colby_2013-2014.pdf
Colby_2011-2012.pdf
Colby_20

Wayne State_2018-2019.pdf
Wayne State_2001-2003.pdf
Wayne State_2011-2013.pdf
Wayne State_2013-2015.pdf
Wayne State_2017-2018.pdf
Wayne State_1993-1995.pdf
Wayne State_2019-2020.pdf
Wayne State_1997-1999.pdf
Wayne State_2007-2009.pdf
Wayne State_1991-1993.pdf
Wayne State_1995-1997.pdf
Wayne State_2005-2007.pdf
Wayne State_2009-2011.pdf
Wayne State_1987-1989.pdf
Great Basin_2007-08.pdf
Great Basin_2019-20.pdf
Great Basin_2017-18.pdf
Great Basin_2006-07.pdf
Great Basin_2016-17.pdf
Great Basin_2008-09.pdf
Great Basin_2003-04.pdf
Great Basin_2013-14.pdf
Great Basin_2015-16.pdf
Great Basin_2005-06.pdf
Great Basin_2018-19.pdf
Great Basin_2009-10.pdf
Great Basin_2004-05.pdf
Great Basin_2001-02.pdf
Great Basin_2002-03.pdf
Great Basin_2011-12.pdf
Great Basin_2010-11.pdf
Great Basin_2012-13.pdf
Great Basin_2014-15.pdf
Great Basin_2020-2021.pdf
Shippensburg_2019-2021.pdf
Shippensburg_2017-2019.pdf
Shippensburg_2013-2015.pdf
Shippensburg_2015-2017.pdf
Shippensburg_2003-2005.pdf
Shippensburg_2007-2

Old Dominion_2010-2011.pdf
Old Dominion_2020-2021.pdf
Old Dominion_2018-2019.pdf
Old Dominion_2006-2008.pdf
Old Dominion_2004-2006.pdf
Shoreline CC_2005-2006.pdf
Shoreline CC_2012-2013.pdf
Shoreline CC_2004-2005.pdf
Shoreline CC_2013-2014.pdf
Shoreline CC_2011-2012.pdf
Shoreline CC_2009-2010.pdf
Shoreline CC_2010-2011.pdf
Shoreline CC_2017-2018.pdf
Shoreline CC_2008-2009.pdf
Shoreline CC_2014-2015.pdf
Shoreline CC_2015-2016.pdf
Shoreline CC_2002-2003.pdf
Shoreline CC_2016-2017.pdf


Shoreline CC_2006-2007.pdf
Shoreline CC_2003-2004.pdf
William Carey_2016-2017.pdf
William Carey_2009-2010.pdf
William Carey_2014-2015.pdf
William Carey_2018-2019.pdf
William Carey_2011-2012.pdf
William Carey_2017-2018.pdf
William Carey_2013-2014.pdf
William Carey_2008-2009.pdf
William Carey_2012-2013.pdf
William Carey_2015-2016.pdf
William Carey_2010-2011.pdf
Bowling Green_Fall2004.pdf
Bowling Green_Fall2015.pdf
Bowling Green_4-2-2020.pdf


Bowling Green_Fall2017.pdf
Bowling Green_Fall2016.pdf
Bowling Green_Fall2003.pdf
Bowling Green_Fall2011.pdf
Bowling Green_Fall2009.pdf
Bowling Green_Fall2008.pdf


Bowling Green_Fall2019.pdf


Bowling Green_Fall2020.pdf


Bowling Green_Fall2002.pdf
Bowling Green_Fall2006.pdf
Bowling Green_Fall2012.pdf
Bowling Green_Fall2007.pdf
Bowling Green_Fall2014.pdf
Bowling Green_Fall2018.pdf
Bowling Green_Fall2005.pdf
Bowling Green_Fall2013.pdf
Bowling Green_Fall2010.pdf
Bowling Green_Spring2019.pdf
Bowling Green_Spring2012.pdf
Bowling Green_Spring2016.pdf
Bowling Green_Spring2018.pdf
Bowling Green_Spring2013.pdf
Bowling Green_Spring2014.pdf
Bowling Green_Spring2001.pdf
Bowling Green_Spring2015.pdf
West Virginia_2012-2013.pdf
West Virginia_1999-2001.pdf
West Virginia_1991-1993.pdf
West Virginia_2010-2011.pdf
West Virginia_2005-2007.pdf
West Virginia_2009-2010.pdf
West Virginia_2007-2009.pdf
West Virginia_2013-2014.pdf
West Virginia_2011-2012.pdf
West Virginia_1986-1987.pdf
West Virginia_1997-1999.pdf
West Virginia_2014-2015.pdf
West Virginia_1995-1997.pdf
West Virginia_2003-2005.pdf
West Virginia_2001-2003.pdf
West Virginia_2017-2018.pdf
West Virginia_1987-1988.pdf
West Virginia_2015-2016.pdf
West Virginia_1993-19

### Sandbox

In [3]:
from itertools import chain
batch_1 = ['Arkansas', 'Bowdoin', 'Ohio State']
batch_2 = ['Colorado - Boulder', 'Minnesota Twin Cities', 'UCLA', 'U Conn']
limit_folders = batch_1 + batch_2
files = [directory + '/' + file
         for directory, _, files in list(os.walk('data'))
         for file in files
         if '.pdf' in file
         and (directory.split('/')[1] not in limit_folders
              if len(limit_folders) != 0
              else True)]

x = get_all_data(files[0])

t = ''.join(list(chain.from_iterable([a['tboxes'] for a in x['pdfminer_detailed']])))
re.findall(r"\n[A-Z\W\s]*\n", t)

NameError: name 're' is not defined

In [4]:
import re

In [8]:
t = ''.join(list(chain.from_iterable([a['tboxes'] for a in x['pdfminer_detailed']])))
re.findall(r"\n[A-Z\W\s]*\n", t)
files[20]

'data/Iowa/Iowa_2013-14.pdf'

In [5]:
cur.execute('select filename from textract')
cur.fetch()

[('UCLA_2017–18.pdf',),
 ('UCLA_2009–10.pdf',),
 ('UCLA_1997–99.pdf',),
 ('UCLA_2007–08.pdf',),
 ('UCLA_1995–97.pdf',),
 ('UCLA_2013–14.pdf',),
 ('UCLA_2001–03.pdf',),
 ('UCLA_2008–09.pdf',),
 ('UCLA_2010–11.pdf',),
 ('UCLA_2016–17.pdf',),
 ('UCLA_2011–12.pdf',),
 ('UCLA_2019–20.pdf',),
 ('UCLA_2018–19.pdf',),
 ('UCLA_1992–93.pdf',),
 ('UCLA_2003–05.pdf',),
 ('UCLA_2012–13.pdf',),
 ('UCLA_1994–95.pdf',),
 ('UCLA_2005–07.pdf',),
 ('UCLA_2014–15.pdf',),
 ('UCLA_1991–92.pdf',),
 ('UCLA_2015–16.pdf',),
 ('UCLA_1993–94.pdf',),
 ('UCLA_1999–2001.pdf',),
 ('U Conn_2008-2009.pdf',),
 ('U Conn_2010-2011.pdf',),
 ('U Conn_2007-2008.pdf',),
 ('U Conn_2015-2016.pdf',),
 ('U Conn_2016-2017.pdf',),
 ('U Conn_2014-2015.pdf',),
 ('U Conn_2006-2007.pdf',),
 ('U Conn_2017-2018.pdf',),
 ('U Conn_1998-1999.pdf',),
 ('U Conn_2005-2006.pdf',),
 ('U Conn_2000-2001.pdf',),
 ('U Conn_2018-2019.pdf',),
 ('U Conn_2012-2013.pdf',),
 ('U Conn_2002-2003.pdf',),
 ('U Conn_2003-2004.pdf',),
 ('U Conn_1999-2000.pdf',)

In [5]:
''.join(list(chain.from_iterable([a['tboxes'] for a in x['pdfminer_detailed']])))[:1000]

'University of Iowa General Catalogs \nOffice of the Registrar \n8-1-2006 \nUniversity of Iowa \nThe University of Iowa General Catalog 2006-2007 \nThe University of Iowa General Catalog 2006-2007 \nFollow this and additional works at: https://ir.uiowa.edu/registrar_catalogs \n Part of the Curriculum and Instruction Commons, and the Higher Education Administration Commons \nCopyright © 2006 The University of Iowa \nHosted by Iowa Research Online. For more information please contact: lib-ir@uiowa.edu. \nGeneral \nCataloG\n2006-07\n2006-07\nUniversity Calendar 2006-2007\n2006\nAugust 21\nSeptember 4\nNovember 20-26\nNovember 23-24\nDecember 8\nDecember 11-15\nDecember 15-16\nDecember 25-26\nDecember 27-January 12 Winter session\nJanuary 1\nFall Semester/Winter Session\nClasses begin\nUniversity holiday\nThanksgiving recess\nUniversity holidays\nClasses end\nExamination week\nCommencement ceremonies\nUniversity holidays\nUniversity holiday\n2007\nJanuary 15\nJanuary 16\nFebruary 25\nMarch

In [10]:
''.join(list(chain.from_iterable([a['characters']['chars'] for a in x['pdfminer_detailed']])))[:1000]

TypeError: sequence item 37: expected str instance, float found

In [7]:
''.join([a['characters']['chars'] for a in x['pdfminer_detailed']][0][:1000])

'University of Iowa General Catalogs \nOffice of the Registrar \n8-1-2006 \nUniversity of Iowa \nThe University of Iowa General Catalog 2006-2007 The University of Iowa General Catalog 2006-2007 \nFollow this and additional works at: https://ir.uiowa.edu/registrar_catalogs \n Part of the Curriculum and Instruction Commons, and the Higher Education Administration Commons \nCopyright © 2006 The University of Iowa Hosted by Iowa Research Online. For more information please contact: lib-ir@uiowa.edu. \n'

In [10]:
import numpy as np
np.array([a['characters']['size'] for a in x['pdfminer_detailed']][0][:1000])

array([11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,  0., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11.,  0., 11., 11., 11., 11., 11.,
       11., 11., 11., 11.,  0., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,  0., 16.,
       16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
       16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
       16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
       16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
       16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
       16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
       16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16

In [12]:
np.array([a['characters']['fontfamily'] for a in x['pdfminer_detailed']][0][:1000])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [13]:
x['fonts']

defaultdict(int,
            {'Roboto-Regular': 0,
             'MetaCondBold-Roman': 1,
             'Futura-CondensedBold': 2,
             'Futura-ExtraBold': 3,
             'Weidemann-Book': 4,
             'Futura-CondensedExtraBold': 5,
             'Weidemann-Bold': 6,
             'Universal-GreekwithMathPi': 7,
             'Weidemann-BookItalic': 8,
             'Futura-Heavy': 9,
             'Futura-Book': 10,
             'Symbol': 11,
             'MetaPlusNormal-Roman': 12})